<a href="https://colab.research.google.com/github/zoraizmohammad/FlavorFlow/blob/main/workflow/stateTransitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#RUN - Has all imports
import pandas as pd
import nltk
from nltk.corpus import words, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [6]:
dataset = pd.read_csv('/content/drive/MyDrive/FlavorFlow/annotated_micro_data.csv')

In [7]:
dataset

,Unnamed: 0,title,ingredients,directions,link,source,NER,ingredients_tokenized,directions_tokenized,ingredient_states,action_states,cuisine,dietary_tags
0,0,No-Bake Nut Cookies,one zero point five zero point five teaspoons ...,"[ `` In a heavy 2-quart saucepan , mix brown s...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': ['boil', 'mix', 'stir'], 'related_...",[9000],"[3, 10, 16]",Other,['None']
1,1,Jewell Ball'S Chicken,one four one one centavo year roentgen short tons,[ `` Place chipped beef on bottom of baking di...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': ['bake', 'mix'], 'related_ingredie...",[552],"[1, 10]",Other,['None']
2,2,Creamy Corn,two sixteen ounces one eight ounces zero point...,"[ `` In a slow cooker , combine all ingredient...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","[{'quantity': None, 'unit': 'two', 'ingredient...","{'actions': ['stir', 'cook'], 'related_ingredi...",[6712],"[16, 5]",Other,['None']
3,3,Chicken Funny,one two ten point five ounces one ten point fi...,"[ `` Boil and debone chicken . `` , `` Put bit...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': ['bake', 'boil'], 'related_ingredi...",[8331],"[1, 3]",Other,['None']
4,4,Reeses Cups(Candy),one zero point seven five one one pound three ...,[ `` Combine first four ingredients and press ...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': [], 'related_ingredients': ['[', '...",[9156],[],Other,['None']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,Pink Fruit Salad,one one one one one two tablespoons,[ `` Combine all ingredients and chill . '' ],www.cookbooks.com/Recipe-Details.aspx?id=183170,Gathered,"[""cherry pie filling"", ""condensed milk"", ""pine...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': [], 'related_ingredients': ['[', '...",[1220],[],Other,['None']
9996,9996,Peppered Steak,one pound one one one twenty-eight ounces zero...,[ `` Cut steaks into strips ; brown in cooking...,www.cookbooks.com/Recipe-Details.aspx?id=462037,Gathered,"[""bell pepper"", ""onion"", ""tomatoes"", ""salt"", ""...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': ['cook', 'serve'], 'related_ingred...",[5542],"[5, 14]",Other,['None']
9997,9997,Chicken Casserole,three pounds one one one point five zero point...,"[ `` Stew and bone fryer . `` , `` Saute in sm...",www.cookbooks.com/Recipe-Details.aspx?id=292083,Gathered,"[""fryer"", ""onion"", ""green pepper"", ""celery"", ""...","[{'quantity': None, 'unit': 'three', 'ingredie...","{'actions': ['saute', 'bake', 'mix', 'cook'], ...",[6086],"[12, 1, 10, 5]",Other,['None']
9998,9998,Sweet Potatoes Casserole,one one one zero point three three three three...,[ `` Mix together for 2 or 3 minutes . Put int...,www.cookbooks.com/Recipe-Details.aspx?id=56276,Gathered,"[""yams"", ""sugar"", ""egg"", ""milk"", ""vanilla flav...","[{'quantity': None, 'unit': 'one', 'ingredient...","{'actions': ['mix'], 'related_ingredients': ['...",[1739],[10],Other,['None']


In [18]:
# Flatten the list of ingredients across all recipes and filter out non-string values
ingredients_list = dataset['NER'].apply(eval).explode()
ingredients_list = ingredients_list[ingredients_list.apply(lambda x: isinstance(x, str))].unique()

# Sorting and generating unique ingredient state mappings
unique_ingredients = sorted(set(ingredients_list))  # Sorting for consistency
ingredient_state_mapping = {ingredient: f"Ingredient_{i+1}" for i, ingredient in enumerate(unique_ingredients)}


2.1.2

In [20]:
# Task 2.1.2: Extract Unique Cooking Steps
# Extracting actions from 'directions_tokenized' where each entry is a dictionary of actions and ingredients

# Flatten the list of actions across all recipes and filter out non-string values
steps_list = dataset['directions_tokenized'].apply(lambda x: eval(x).get('actions', [])).explode()
steps_list = steps_list[steps_list.apply(lambda x: isinstance(x, str))].unique()

# Sorting and generating unique step state mappings
unique_steps = sorted(set(steps_list))
step_state_mapping = {step: f"Step_{i+1}" for i, step in enumerate(unique_steps)}

# Displaying the step state mapping
step_state_mapping


{'bake': 'Step_1',
 'blend': 'Step_2',
 'boil': 'Step_3',
 'chop': 'Step_4',
 'cook': 'Step_5',
 'dice': 'Step_6',
 'fry': 'Step_7',
 'grill': 'Step_8',
 'mince': 'Step_9',
 'mix': 'Step_10',
 'roast': 'Step_11',
 'saute': 'Step_12',
 'season': 'Step_13',
 'serve': 'Step_14',
 'slice': 'Step_15',
 'stir': 'Step_16',
 'whisk': 'Step_17'}

2.1.3

In [21]:
# Task 2.1.3: Create a Comprehensive State Mapping
# Combining the ingredient and step mappings into a single dictionary

state_mapping = {
    'ingredients': ingredient_state_mapping,
    'steps': step_state_mapping
}

# Displaying the combined state mapping for reference
state_mapping


{'ingredients': {"'s Cajun": 'Ingredient_1',
  "'s Crabmeat": 'Ingredient_2',
  "'s applesauce": 'Ingredient_3',
  "'s cheese": 'Ingredient_4',
  "'s chili beans": 'Ingredient_5',
  "'s country": 'Ingredient_6',
  "'s ketchup": 'Ingredient_7',
  "'s meat marinade": 'Ingredient_8',
  "'s sauce": 'Ingredient_9',
  "'s sausage": 'Ingredient_10',
  "'s secret ingredients": 'Ingredient_11',
  '*': 'Ingredient_12',
  '+': 'Ingredient_13',
  ', washed': 'Ingredient_14',
  'A.': 'Ingredient_15',
  'Accent': 'Ingredient_16',
  'Accent salt': 'Ingredient_17',
  'Accent seasoning': 'Ingredient_18',
  'Active Dry Yeast': 'Ingredient_19',
  'Adams wheat beer': 'Ingredient_20',
  'Alaga syrup': 'Ingredient_21',
  'Alfredo Sauce': 'Ingredient_22',
  'All-Bran': 'Ingredient_23',
  'All-Bran cereal': 'Ingredient_24',
  'All-Purpose Flour': 'Ingredient_25',
  'Allspice': 'Ingredient_26',
  'American cheese': 'Ingredient_27',
  'American cheese slices': 'Ingredient_28',
  'Angel': 'Ingredient_29',
  'Arb

2.2.1, 2.2.2, 2.2.3

In [27]:
from collections import defaultdict

# Task 2.2.1 and 2.2.2: Loop Through Recipes and Record State Transitions

# Initialize a dictionary to hold transition frequencies
transition_frequency = defaultdict(int)

# Loop through each recipe in the dataset
for index, row in dataset.iterrows():
    # Retrieve the steps and ingredients for the recipe, if available
    ingredients = row['NER'] if 'NER' in row else []
    actions = eval(row['directions_tokenized']).get('actions', []) if 'directions_tokenized' in row else []

    # Combine ingredients and actions into a sequence of states
    ingredient_sequence = [ingredient_state_mapping.get(ing, None) for ing in eval(ingredients) if ing in ingredient_state_mapping]
    action_sequence = [step_state_mapping.get(act, None) for act in actions if act in step_state_mapping]

    # Create a full sequence by interleaving ingredients and actions (assuming order matters as provided)
    state_sequence = ingredient_sequence + action_sequence

    # Record transitions between consecutive states
    for i in range(len(state_sequence) - 1):
        if state_sequence[i] and state_sequence[i+1]:  # Ensuring no None values
            transition = (state_sequence[i], state_sequence[i + 1])
            transition_frequency[transition] += 1

# Task 2.2.3: Convert the transition frequency dictionary to a DataFrame for analysis
transition_df = pd.DataFrame(list(transition_frequency.items()), columns=['Transition', 'Frequency'])
transition_df[['From_State', 'To_State']] = pd.DataFrame(transition_df['Transition'].tolist(), index=transition_df.index)
transition_df.drop(columns='Transition', inplace=True)

transition_df.head()


,Frequency,From_State,To_State
0,16,Ingredient_1028,Ingredient_2744
1,129,Ingredient_2744,Ingredient_3900
2,63,Ingredient_3900,Ingredient_2860
3,12,Ingredient_2860,Ingredient_1042
4,1,Ingredient_1042,Ingredient_896
